<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup-Data-Pre-Processing" data-toc-modified-id="Setup-Data-Pre-Processing-0"><span class="toc-item-num">0&nbsp;&nbsp;</span>Setup Data Pre-Processing</a></span><ul class="toc-item"><li><span><a href="#Raw-Data-to-Feature-Data" data-toc-modified-id="Raw-Data-to-Feature-Data-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Raw Data to Feature Data</a></span></li><li><span><a href="#Tensor-Data-Structure-Input" data-toc-modified-id="Tensor-Data-Structure-Input-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>Tensor Data-Structure Input</a></span></li></ul></li><li><span><a href="#Architecture-Basics" data-toc-modified-id="Architecture-Basics-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Architecture Basics</a></span><ul class="toc-item"><li><span><a href="#Gated-CNN" data-toc-modified-id="Gated-CNN-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Gated CNN</a></span><ul class="toc-item"><li><span><a href="#GCNN-2D" data-toc-modified-id="GCNN-2D-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>GCNN 2D</a></span></li></ul></li></ul></li></ul></div>

In [1]:
# Imports e definicoes auxiliares, TODO - fazer module apenas para estas definicoes auxiliares
from IPython.display import Math
#!pip install librosa
import tensorflow as tf
from tensorflow.python import debug as tf_debug
from tensorflow.python import util
from tensorflow.python.framework import ops as tf_ops
tf.summary.initialize = tf.contrib.summary.initialize
tf.variable_scope = tf.compat.v1.variable_scope

#  Importing helper functions for spectrogram, comp graph viz and helper functions to calculate filters and device scopes
from spect import *
from cgraphviz import *
from tfhelper import *
from dataloader import *


import numpy as np
from six import string_types
import argparse
import functools
from numpy import asarray, array, ravel, repeat, prod, mean, where, ones
import sys
import copy
import re
import os

import matplotlib.pyplot as plt
from tensorflow.python.client import timeline
    
# In TensorFlow a device name is a string of the following form:
#   /job:<name>/replica:<replica>/task:<task>/device:<type>:<device_num>
#
# <name> is a short identifier conforming to the regexp
#     [a-zA-Z][_a-zA-Z]*
# <type> is a supported device type (e.g. 'cpu' or 'gpu')
# <replica>, <task>, <device_num> are small non-negative integers and are
# densely allocated (except in tests).
#
# For some purposes, we also allow device patterns, which can specify
# some or none of the specific fields above, with missing components,
# or "<component>:*" indicating "any value allowed for that component.
#
# For example:
#   "/job:param_server"   - Consider any devices in the "param_server" job
#   "/device:cpu:*"       - Consider any cpu devices in any job/task/replica
#   "/job:*/replica:*/task:*/device:cpu:*"  - Consider any cpu devices in any
#                                             job/task/replica
#   "/job:w/replica:0/task:0/device:gpu:*"  - Consider any gpu devices in
#                                             replica 0, task 0, of job "w"

import tensorflow as tf
tf.variable_scope = tf.compat.v1.variable_scope
import numpy as np
from cgraphviz import *  
from tfhelper import *
from ckpt_hookers import *
from dataloader import *
import argparse
import importlib
import dataloader


class AttrDict(dict):
    
  __getattr__ = dict.__getitem__
  __setattr__ = dict.__setitem__      

W0826 16:12:05.568207 139844138194304 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

/home/penalvad/miniconda3/envs/scicomptf1_37/lib/python2.7/site-packages/librosa/__init__.py:40: DeprecationWarning: You are using librosa with Python 2. Please note that librosa 0.7 will be the last version to support Python 2, after which it will require Python 3 or later.
  DeprecationWarning)
W0826 16:12:06.059977 139844138194304 deprecation_wrapper.py:119] From ckpt_hookers.py:7: The name tf.train.SessionRunHook is deprecated. Please use tf.estimator.SessionRunHook instead.

W0826 16:12:06.060863 139844138194304 deprecation_wrapper.py:119] From ckpt_hookers.py:19: Th

In [2]:
dataload = Sass('/home/penalvad/stattus4/dnn_test_data/', num_samples=8, number_of_batches=1, split_tax=.9, freq_size=600, time_size=80, same_data_validation=True)

# Setup Data Pre-Processing 

## Raw Data to Feature Data

In [ ]:
# Raw data to Log-Spectogram 
# Read data from the hard-drive
spect_list = []
raw_list = []
nomes = [f for f in os.listdir('/home/penalvad/stattus4/benchfull/')]
label_list = []
c = 0
b = 0 
m = 0
for i in range(len(nomes)):
    #ignore .mat
  if nomes[i].find('.mat') == -1 and (nomes[i].find('sv') != -1 or nomes[i].find('SV') != -1) and c < 50:    
    
    data,fs = librosa.load('/home/penalvad/stattus4/benchfull/'+nomes[i],sr=None)
    spec = subbed_spect(data,fs,plot=False)
    m = np.shape(spec[0])
    
    if m[0] > 600 and m[1] > 50:      
      spect_list.append( spec[0][:600,:50] )
      label_list.append(nomes[i][0:2])
      c += 1        
    #raw_list.append((data,fs))
    

  if nomes[i].find('.mat') == -1 and (nomes[i].find('cv') != -1 or nomes[i].find('CV') != -1 )and b < 50:
    
    data,fs = librosa.load('/home/penalvad/stattus4/benchfull/'+nomes[i],sr=None)
    spec = subbed_spect(data,fs,plot=False)
    m = np.shape(spec[0])
    
    if m[0] > 600 and m[1] > 50:
      spect_list.append( spec[0][:600,:50] )
      label_list.append(nomes[i][0:2])
      b += 1             
    #raw_list.append((data,fs))    

## Tensor Data-Structure Input

In [ ]:
# Data to Tensor Data for consumption on Tensorflow models
# TODO - equalize data size for downsampled raw data
# - On supposition that the data size is the same for all time series, if not is necessary to assert it.
# - The same is true for spect, however downsampled spect may lead to smaller ts.
#ts  = sample_gaussmix(shape=(10000,), mean_base=0.5, var_base=0.15, random_mods=True, num_mods=50)

batch = len(spect_list)
#data_size_ts = len(raw_list[0][0])
data_size_image = (len(spect_list[0][:,0]),len(spect_list[0][0,:]))
channels = 1
#print(data_size_ts)

#data_tensor_1D = np.ndarray( shape=(batch,data_size_ts,channels) )
data_tensor = np.ndarray( shape=(batch,data_size_image[0],data_size_image[1],channels) )

for i in range(batch):

  #data_tensor_1D[i,:,0] = raw_list[i][0]
  data_tensor[i,:,:,0] = spect_list[i][:,:]
    
np.shape(data_tensor)

In [ ]:
print( dtrain[0][1][2]  ) 

In [3]:
dtrain = dataload.training()
dtest = dataload.testing()

data_train = np.ndarray( shape=(14,600,80,1) )
data_test = np.ndarray( shape=(8,600,50,1) )

for i in range(7):
    
  data_train[2*i,:,:,0] = dtrain[i][0][2]
  data_train[2*i+1,:,:,0] = dtrain[i][1][2]

#for i in range(4):

  #data_test[2*i,:,:,0] = dtest[i][0][2]
  #data_test[2*i+1,:,:,0] = dtest[i][1][2]

In [5]:
np.shape(data_train)

(14, 600, 80, 1)

# Architecture Basics

In [4]:
# Definitions: Build computation graph and run session
####
### General purpose builder function, to be substituted in FUTURE refatoration mentioned above
## Serve as Computation Graph builder and saver (through session saver), editor and Computational Graph runner.
#
class Builder:  
  
  def __init__(self, **kwargs):
    '''
      Build a Tensorflow computational graph, from scratch.      
        
      Requires signal input parameters such:
      - batch: for batch size
      - datasize: give a tuple with the signal input shape
      - channels: self explain.
      
      
      Note 0: You can always call Builder.get_directives(archblocktype) to know the inner params to the building block architecture.
      Note 1: The object guard reference for each op in a dictionary namescopes with the name of each block.
      TODO - export serialized file containing the model (e.g. .pb)
    '''  
    
    #Use attribute dict ##FUTURE: use arch block class with enums specifing the blocks
    
    #Provisional name tracking for this network.
    
    self.namescopes = AttrDict()    
    self.architectures = AttrDict()
    self.architectures.gcnn2d = self._gcnn2d
    self.architectures.reducemean = self._reducemean
    self.architectures.losscrossentropy = self._losscrossentropy
    self.architectures.softmax = self._softmax
    self.architectures.maxpooling2d = self._maxpooling2d
    self.architectures.cnn2d = self._cnn2d
    
    
    #TODO also use attribute dict, each arch has it own directive dict in with each directive has its shape has value
    ##FUTURE: use arch block class with enums specifing the blocks
    ##   Auto compile directives from json and query parameters,
    ## return requisition or error if parameter not given or is not from appropriated shape and type. 
    self.directives = {
        'gcnn2d': {'channels_out':1,
                   'filter_size':2},
        'reducemean':{'None':None},
        'losscrossentropy':{'num_labels':1,
                            'batch':1,
                            'learningrate':1},
        'softmax':{'num_labels':1},  
        'residualgcnn1d':{'channels_out':1,
                          'filter_size':1},
        'gcnn1d':{'channels_out':1,
                   'filter_size':2
                 },
        'cnn2d':{'channels_out':1,
                 'filter_size':2            
                },
        'maxpooling2d':{'poolsize':2                                       
        },
        'signalimg':{'batch':1,
                   'datasize':2,
                   'channels':1,
                   'dtype':1
                    }    
    }
    
    tf.reset_default_graph()
    # TODO-Define Specific Exception Handler for the Input
    try:
        
      self.batch = kwargs['batch']
      self.datasize = kwargs['datasize']
      self.channels = '' or kwargs['channels']
      self.dtype = kwargs['dtype']
    
    except KeyError:
        
      sys.exit('Signal Input Not Defined Error')      
    
    try:
        
      self.num_input = kwargs['num_input']
    
    except:
        
      self.num_input = 1
      
    self.graph = tf.Graph()
    self.signal_in = []  
    # FUTURE: this will go to another class
    self.arch_blocks = {}    
    
  def __call__(self, **kwargs):        
   
    try:
      isinput = kwargs['isinput']
      if type(isinput) != type(bool()):
        isinput = True
    except:
      isinput = False
    
    try:
      lastnamescope= kwargs['lastnamescope']
      if type(lastnamescope) != type(bool()):
        lastnamescope= False
    except:
      lastnamescope= True       


    if isinput:
      with self.graph.as_default():
      # TODO - Different input shapes and data types (char,string,hexa,float,int...)                      
        if self.num_input > 0:
          self.signal_in.append(tf.placeholder(self.dtype,shape=(None,self.datasize[0],self.datasize[1],self.channels),name='signal_in'))           
          self.namescopes['signal_in'] = self.signal_in
          self.num_input -= 1
        
      return self.signal_in[-1]
    elif lastnamescope:
      with self.graph.as_default():  
        return self.graph.get_tensor_by_name(self.graph.get_operations()[-1].name+':0')
    else:
      try:
        namescope = kwargs['namescope']
        with self.graph.as_default():
          with tf.name_scope(namescope+'/'):
            opa = 0
            for op in self.graph.get_operations():
              if op.name.split('/')[0] == namescope:
                opa = op 
            return self.graph.get_tensor_by_name(opa.name+':0')
      except:        
        sys.exit("Namescope not given to the input of architectures")            
        
  def _compute_membatchsize(self):
    c = 0
    for key in self.namescopes.keys():
      for tensor in self.namescopes[key]:          
        if type(tensor).__name__ == 'Tensor':
          
          c = c + tensor.get_shape().num_elements()          

    return c*8.0/(1024.0*1024.0*1024.0)  

  def _compute_memusage(self):

    c = 0      
    for v in buildgraph.graph._collections['variables']:      
      c = c + v.shape.num_elements()
    return c*4.0/(1024.0*1024.0*1024.0)      
        
  def estimate_memusage(self,limit = 12.0):
    self.mem_usage = self._compute_membatchsize() + self._compute_memusage()
    if self.mem_usage > limit:
      raise MemoryError('The model will allocate more memory, {}, than the given limit.'.format(self.mem_usage))    
    
  def assertive(self, archname):
    
    if archname == 'maxpooling2d':
      if hasattr(self, 'pool') != True or getattr(self, 'pool') == None:
        raise NameError('{} pools not setted'.format(archname))  
      if hasattr(self, 'numblock') != True or getattr(self, 'numblock') == None:
        raise NameError('{} block number not setted'.format(archname))      
        
    else:
      if hasattr(self, 'deepness') != True or getattr(self, 'deepness') == None:
        raise NameError('{} deepness not setted'.format(archname))
    
      if hasattr(self, 'numblock') != True or getattr(self, 'numblock') == None:
        raise NameError('{} block number not setted'.format(archname))    

  def _gcnn2d(self, **kwargs):
    try:
        
      channels_out = kwargs['channels_out']
      filter_size = kwargs['filter_size']
          
    except KeyError:
        
      sys.exit('Parameters Not Defined Error')  
    
    signal_in = self(**kwargs)
     
    ####            
    ## Keras convolutions. Classes, so dont behave like functions but outputs Tensor, use its functions to query the variables filters and bias                    
    self.assertive('gcnn2d')
    
    with self.graph.as_default():
      with tf.variable_scope('gcnn2d'+self.numblock+self.deepness[0]):
        
        self.namescopes['gcnn2d'+self.numblock+self.deepness[0]] = []    
        
        with self.graph.device(dev_selector(arg1='foo')('gcnn2d')):
          conv_linear = tf.keras.layers.Conv2D( channels_out, filter_size, padding='valid', name='conv_linear', use_bias=True, kernel_initializer=tf.initializers.lecun_normal(seed=137), bias_initializer=tf.initializers.lecun_normal(seed=137) )(signal_in)
          self.namescopes['gcnn2d'+self.numblock+self.deepness[0]].append(conv_linear)            

        with self.graph.device(dev_selector(arg1='foo')('gcnn2d')):
          conv_gate = tf.sigmoid(tf.keras.layers.Conv2D( channels_out, filter_size, padding='valid', name='conv', use_bias=True,  kernel_initializer=tf.initializers.lecun_normal(seed=137), bias_initializer=tf.initializers.lecun_normal(seed=137) )(signal_in),name='conv_sigmoid')          
          self.namescopes['gcnn2d'+self.numblock+self.deepness[0]].append(conv_gate)  
            
        with self.graph.device(dev_selector(arg1='foo')('gcnn2d')):
          gated_convolutions = tf.multiply(conv_linear,conv_gate,name='gated_convolutions') 
          self.namescopes['gcnn2d'+self.numblock+self.deepness[0]].append(gated_convolutions)
        
        self.deepness.pop(0)
        if len(self.deepness) == 0:
          self.deepness = None  
        
  def _cnn2d(self, **kwargs):
    
    try:
        
      channels_out = kwargs['channels_out']
      filter_size = kwargs['filter_size']          
        
    except KeyError:        
        
      sys.exit('Parameters Not Defined Error')  
    
    signal_in = self(**kwargs)
     
    ####            
    ## Keras convolutions. Classes, so dont behave like functions but outputs Tensor, use its functions to query the variables filters and bias                    
    self.assertive('cnn2d')
    
    with self.graph.as_default():
      with tf.variable_scope('cnn2d'+self.numblock+self.deepness[0]):
        
        self.namescopes['cnn2d'+self.numblock+self.deepness[0]] = []    
        
        with self.graph.device(dev_selector(arg1='foo')('cnn2d')):
          conv_linear = tf.keras.layers.Conv2D( channels_out, filter_size, padding='valid', name='conv_linear', use_bias=True, kernel_initializer=tf.initializers.lecun_normal(seed=137), bias_initializer=tf.initializers.lecun_normal(seed=137), activation=None )(signal_in)
          lrelu = tf.keras.layers.LeakyReLU(alpha=0.3)(conv_linear)          
          self.namescopes['cnn2d'+self.numblock+self.deepness[0]].append(lrelu)            
        
        self.deepness.pop(0)
        if len(self.deepness) == 0:
          self.deepness = None      
        
  def _maxpooling2d(self, **kwargs):
    try:
      poolsize = kwargs['poolsize']
    except:
      poolsize = (2,2)
    
    
    signal_in = self(**kwargs)
    
    self.assertive('maxpooling2d')
    with self.graph.as_default():        
      with tf.variable_scope('maxpooling2d'+self.numblock+self.pool[0]):  
        
        self.namescopes['maxpooling2d'+self.numblock+self.pool[0]] = []
        maxpool = tf.keras.layers.MaxPool2D( pool_size=poolsize, strides=None, padding='valid', data_format=None)(signal_in)
        self.namescopes['maxpooling2d'+self.numblock+self.pool[0]].append(maxpool)
        
    self.pool.pop(0)
    if len(self.pool) == 0:
      self.pool = None      

  def _softmax(self, **kwargs):        

    try:        
      num_labels = kwargs['num_labels']      
    except Exception:        
      sys.exit('Parameters Not Defined Error')       
    
    signal_in = self(**kwargs)    
    self.assertive('softmax')   
        
    with self.graph.as_default():
      with tf.variable_scope('softmax'+self.numblock+self.deepness[0]):
        
        self.namescopes['softmax'+self.numblock+self.deepness[0]] = []          
        
        with self.graph.device(dev_selector(arg1='foo')('softmax')):
          logits = tf.contrib.layers.fully_connected(signal_in, num_labels, activation_fn=None, normalizer_fn=None, normalizer_params=None, weights_initializer=tf.initializers.lecun_normal(seed=731), weights_regularizer=None, biases_initializer=tf.initializers.lecun_normal(seed=777), biases_regularizer=None, reuse=None, variables_collections=None, outputs_collections=None, trainable=True, scope='logit')
          self.namescopes['softmax'+self.numblock+self.deepness[0]].append(logits)
            
        with self.graph.device(dev_selector(arg1='foo')('softmax')):
          softmax = tf.nn.softmax(logits,axis=0)            
          self.namescopes['softmax'+self.numblock+self.deepness[0]].append(softmax)                      
        
        self.deepness.pop(0)
        if len(self.deepness) == 0:
          self.deepness = None         

  def _losscrossentropy(self, **kwargs):    
    
    try:        
      num_labels = kwargs['num_labels']
      batch = 10 #kwargs['batch']   
    except Exception:        
      sys.exit('Parameters Not Defined Error')
    # TODO SIGNAL IN de acordo com arquitetura.
    signal_in = self(**kwargs)            
    ####        
    ## Keras convolutions. Classes, so dont behave like functions but outputs Tensor, use its functions to query the variables filters and bias   
    self.assertive('losscrossentropy') 
    with self.graph.as_default():
      self.learning_rate = tf.placeholder(tf.float32,shape=(),name='learning_rate')
    
      with tf.variable_scope('losscrossentropy'+self.numblock+self.deepness[0]):        

        self.namescopes['losscrossentropy'+self.numblock+self.deepness[0]] = []        
        self.l_vec = tf.placeholder(tf.float32,(None,num_labels), name='labels')
        with self.graph.device(dev_selector(arg1='foo')('losscrossentropy')):
          self.loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False, label_smoothing=0)(self.l_vec, signal_in)
          self.namescopes['losscrossentropy'+self.numblock+self.deepness[0]].append(self.loss)
           

        with self.graph.device(dev_selector(arg1='foo')('losscrossentropy')):            
          self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate) 
          #self.optimizer = RAdam(learning_rate=0.005, min_lr=0.0002 ,total_steps=30)
          self.minimize_op = self.optimizer.minimize(self.loss)
          self.namescopes['losscrossentropy'+self.numblock+self.deepness[0]].append(self.optimizer)
          self.namescopes['losscrossentropy'+self.numblock+self.deepness[0]].append(self.minimize_op)
            
        self.init_op = tf.global_variables_initializer()            
        self.saver = tf.train.Saver()
        
        #if 'limit' in kwargs:          
          #self.estimate_memusage(limit = kwargs['limit'])
        #else:
          #self.estimate_memusage()
  
        self.deepness.pop(0)
        if len(self.deepness) == 0:
          self.deepness = None
        
  def _reducemean(self, **kwargs):
     
    scope_tensor_name = find_softmax(self.graph)
    inputs = get_tensor_list_from_scopes(self.graph, scope_tensor_name)    
    
    self.assertive('reducemean')
    
    with self.graph.as_default():
      with tf.variable_scope('reducemean'+self.numblock+self.deepness[0]):

        self.namescopes['reducemean'+self.numblock+self.deepness[0]] = []
  
        with self.graph.device(dev_selector(arg1='foo')('reducemean')):
          rm = tf.math.reduce_mean( inputs, axis=1 )
          self.namescopes['reducemean'+self.numblock+self.deepness[0]].append(rm)
          rmshape = tf.shape(rm)
          rmean = tf.reshape(rm, shape=(rmshape[0], rmshape[2]))   
          self.namescopes['reducemean'+self.numblock+self.deepness[0]].append(rmean)
            
        self.deepness.pop(0)
        if len(self.deepness) == 0:
          self.deepness = None
  
  def get_directives(self,arch):
      
    # TODO: Use as automatic requirement builder from file, maybe json.
    # Provisory setup for namespace tags
    #self._nametags = {arch:{}}
    return self.directives[arch]     
      
  # provisory configuration for next sequence of build module names, FUTURE: Hyper-Param-Protocol will learn
# the names automatically from given configuration or from the graph architecture updating.
  def config_block_name(self, deepness=0, numblock=0, pool = -1):
    '''
      Helper function to configure block architecture names.
      Inputs are deepness of the block, if there is any hidden layer, and the block number if there is more than one block with this architecture in the graph.          
    '''
    if pool < 0:
      self.pool = ['']    
    else:  
      self.pool = ['']
      for i in range(pool):
        self.pool.append('_p'+str(i))
    
    if deepness < 0:      
      self.deepness = ['']
    else:  
      self.deepness = ['']
      for i in range(deepness):
        self.deepness.append('_d'+str(i))
    if numblock <= 0:
      self.numblock = ''
    else:
      self.numblock = '_'+str(numblock)
        
  # Definitions: Build computation graph and run session
  def build_graph_module(self, arch, show_cgraph=True, verbose=False, **kwargs):
    '''
      Build a Tensorflow computational graph.building_function
      Currently supports 1-D gcnn or residual 1-D gcnn for unidimensional variable and 2D gcnn for bidimensional frame data.
    
      tensor_scope_name:  name scope of the architecture unit to build the next module
    
        
      kwargs:    
      filter_size: conv filter size. Scalar number for time series, bidimensional tuple for image.
      channels_out: Number of channels for output convolution.
    
      TODO - export serialized file containing the model (e.g. .pb)
    '''
    
    self.architectures[arch](**kwargs) 
    
    with self.graph.as_default(): 
      if show_cgraph:
        print_display_cgraph(self.graph, verbose=verbose)
        
  def _save_trainable_vars(self, blockname):
    with self.graph.as_default():   
        
        
      saver = tf.train.Saver(var_list=self.graph.get_collection('trainable_variables') )      
      tf.global_variables_initializer()
        
      sess = tf.Session(graph=self.graph)      
      sess.run(self.graph.get_operations()[-1])   
      saver.save(sess, os.getcwd()+'/'+blockname)
      #saver.export_meta_graph(filename=blockname+'.constructor', collection_list='trainable_variables', export_scope=None, strip_default_attrs=False)
      self.arch_blocks[blockname] = blockname
        
  def define_block(self,blockname):             
    
    self._save_trainable_vars(blockname)    
    
    graph = tf.Graph()
    with graph.as_default():                                      
      sess = tf.Session(graph=graph)  
      
      new_saver = tf.train.import_meta_graph(os.getcwd()+'/'+self.arch_blocks[blockname]+'.meta', import_scope = blockname)             
      new_saver.restore(sess, os.getcwd()+'/'+self.arch_blocks[blockname])           
      ####  
      ## Redefinir signal_in dos blocos utilizados para construir este bloco através do dict namescopo
      ## para que run_cgraph consiga achar inputs  
      # 
        
      ####
      ## Signalin handling provisorio, fazer um modulo apenas para lidar com signal
      #        
      self.graph = graph
      self.signal_in = []
      self.signal_in.append(self.graph.get_tensor_by_name(blockname+'/signal_in:0'))   

  # Build block from name in the top level of the blocks, this is a previous version that take names inside namescope dict and build new block  
  def from_block(self,archblockname,tag = ""):
        
    with self.graph.as_default():              
      sess = tf.Session(graph=self.graph)  

      new_saver = tf.train.import_meta_graph(os.getcwd()+'/'+self.arch_blocks[archblockname]+'.meta', import_scope = archblockname+tag)        
      new_saver.restore(sess, os.getcwd()+'/'+self.arch_blocks[archblockname])            
      
      self.signal_in.append(self.graph.get_tensor_by_name(archblockname+tag+'/signal_in:0'))
      self.num_input -= 1        

    
  def run_cgraph(self, feed_dict, op_to_run = '', number_of_runs = 10, mode = 'feed forward', auto_lr = True, new_session=True, output_log = True, verbose=True):
    '''
     Run Session
     TODO - session log output
     TODO - in the mode feed forward, only get tensor value for input, in the mode minimize run the last op that represents minimization     
     TODO - make op to run optional depending on modes
     TODO - getvariable by namescope
    '''       
    if new_session:
      session = tf.Session(graph=self.graph)  
      #session = tf_debug.TensorBoardDebugWrapperSession(session,"grpc://localhost:6064")
    else:
      session = tf.get_default_session()               

    with session.as_default():
    #with tf.device(_dev_selector(''))
      feed_dict = feed_dict
      options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
      run_metadata = tf.RunMetadata()
      opval = []
      self.lossval = np.zeros(number_of_runs)
          
    # Search for Initialization: TODO binary search, using brute force ... 
      session.run(self.init_op)
      print('Global Variables Initialized')
        
    # Perform  gradient descent steps or feed forward according to the Operation Required   
      
      self.op_to_run = op_to_run
      #with tf.summary.FileWriter('/home/penalvad/stattus4/stattus4-audio-models/notebooks/',graph=self.graph,session=session) as writer:
        #tf.summary.initialize(graph=self.graph,session=session)               
             
      for step in range(number_of_runs):
        
        if mode == 'minimize':
            
          loss_value, learning_rate,_ = session.run([self.loss, self.learning_rate, self.minimize_op], feed_dict=feed_dict)
          self.lossval[step] = loss_value
            
          if step == 0:  
            self.lr = learning_rate  
                        
          if step % 1 == 0 and verbose:        
            print("Step:", step, " Loss:", self.lossval[step])    
            
          K = 3
          p1 = 0.98
          p2 = 2**(-6)
          
          gamma0 = 1.2
          minrate = 0.0001
          maxrate = 0.03       
          scale = 0.1
            
          if auto_lr and step >= K:
            vect = self.lossval[step-K:step]
            thresh =  np.mean(self.lossval[:K]) - np.mean(2.*np.abs(np.diff( self.lossval[:K] ))) 
            firstmean = (( np.mean(vect)- p2 )/( p1 - p2 )-0.5)*2.0
            firstdiff =  ( np.mean( np.abs(np.diff(vect)) - np.min(np.abs(np.diff(vect))) )/( np.max(np.abs(np.diff(vect))) - np.min(np.abs(np.diff(vect))) + 1e-2)-0.5)*2.0
            
            learning_rate = learning_rate*gamma0**firstmean
            slope = np.mean(vect)
            
            #print('slope ', slope)
            #print('the normed mean: ',firstmean)
            #print('mean difference ',firstdiff)
            #print('diff ', np.mean( np.abs(np.diff( self.lossval[step:step+K] ))) )
            #print('thresh ', 2.*thresh)
            
            if (slope > 0.001 or slope < thresh) and (np.mean( np.abs(np.diff( self.lossval[step:step+K] ))) > 0.001) and learning_rate >= minrate:
              learning_rate = learning_rate*gamma0**( -np.sqrt(firstdiff**2.0) )   
              print('down')
            else:
              learning_rate = learning_rate*( gamma0**(-np.arctan(firstdiff) ))
              print('up')
            
            if learning_rate < minrate: 
              learning_rate = minrate
            elif learning_rate > maxrate:
              learning_rate = maxrate                
            
            #learning_rate = 0.01*(1.0-firstdiff)/(1.0 + firstdiff*step)
            feed_dict[self.learning_rate]= learning_rate
            print('learning rate ', learning_rate)
            
          else:
            
            if step > K-1:                                         
                
              ord = np.mean(np.abs(np.diff( self.lossval[step-K:step] )))  
              print(ord)
              ord = ffff(ord, self.lossval[step])
              print('loss drop ', ord)

              # ANDERSON SUGESTÃO
              
              if self.lossval[step] < 0.0:
                  print('anderson')
                  gain = 1e-5
                  amort = 1e-5
                  lr = gain*(self.lossval[step])**2./(ord+amort)
                  capHigh = 7e-4
                  capLow = 5e-7
              
                  if lr > capHigh:
                      lr = capHigh
                  elif lr < capLow:
                      lr = capLow              
              else:
                  print('penalva')
                  lr = ( ((np.log2(step*0.01+2.0)*self.lr)**(1./(np.log2(step*0.1+1.0)+1.0)))/( ( self.lossval[step]//(self.lossval[step]*0.05) )*(1.0 + step*(ord//(0.002) ) ) ) )%0.001
                  lr = (lr + (lr // 0.0000005) )% 0.0003 + 0.0000005
              feed_dict[self.learning_rate]= lr
              print('learning rate ', lr)
              if self.lossval[step] < self.lossval[step-1] and self.lossval[step] < 0.001:
                print('salvando modelo...')                  
                self.saver.save(session, './ckpt/model')
              print(3*'\n')                  
            
            else:
                
              feed_dict[self.learning_rate]= (np.log2(step*0.01+2.0)*self.lr)**(1./(np.log2(step*0.01+1.0)+1.0))
              print('learning rate ', (np.log2(step*0.01+2.0)*self.lr)**(1./(np.log2(step*0.01+1.0)+1.0)))
              print(3*'\n')        

          
        if self.op_to_run != '':
          op_value = session.run([self.op_to_run], feed_dict=feed_dict)
          opval.append( op_value )
          print(op_value)   

    return opval                          

##  Gated CNN
  Gated CNN is a doubled CNN in whom one of the convoluted signals does the role of opening/closing the network, giving an **Attention Mechanism** to the convolution, for being activated by a sigmoid.
  It gives non-vanishing gradient, since the multiplication rule for the derivative applies, also, applies gradient to the linear convoluted part.
  

### GCNN 2D
  This unit uses sigmoid function in all frame pixels and do element-wise multiplication. Use it for images or time-series TF Features frame. The standard convolution mode is Valid. The output size of the convolution is given by:
 
 Padding Valid
\begin{align}
dimemsion output size = \ceil{ \frac{dim size - (kernel dim size - 1)*dilation rate}{stride} }
\end{align} 

Where dilation rate is how may pixel are each filter spaced from each other, when dilation rate >= 1 then stride = 1 (since if you stride will be losing information from the data).

In [9]:
#### Hand  filter size an deepness
##
#
'''
filterlist = []

filterlista = [3,3,3,3]
filterlist.append(filterlista)
a = calculatefilter(600, filterlista)

# Max pooling

filterlistb = [5,5,5,5]
filterlist.append(filterlistb)
b = calculatefilter(a/3.0, filterlistb)

# Max pooling

filterlistc = [7,7,7,7]
filterlist.append(filterlistc)
c = calculatefilter(b/2., filterlistc)

# Max pooling 

filterlistd = [5,5,5,5]
filterlist.append(filterlistd)
d = calculatefilter(c/2.0, filterlistd)


# Max pooling 

filterliste = [5,4]
filterlist.append(filterliste)
e = calculatefilter(d/2.0, filterliste)

#print('First')
#print(e)

################################## 
##################################
##################################
##################################

filterlist = []

filterlista = [3,3,3,3]
filterlist.append(filterlista)
a = calculatefilter(600, filterlista)

# Max pooling

filterlistb = [5,5,5,5,5,5,5]
filterlist.append(filterlistb)
b = calculatefilter(a/2.0, filterlistb)

# Max pooling, Gate here

filterlistc = [7,7,7,7,7,7,7]
filterlist.append(filterlistc)
c = calculatefilter(b/2., filterlistc)

# Max pooling 

filterlistd = [5,5,5,5]
filterlist.append(filterlistd)
d = calculatefilter(c/2.0, filterlistd)


# Max pooling 

filterliste = [5,4,7]
filterlist.append(filterliste)
e = calculatefilter(d/2.0, filterliste)

print('Second')
print(e)

################################## 
##################################
##################################
##################################

filterlist = []

filterlista = [3,3,3,3]
filterlist.append(filterlista)
a = calculatefilter(600, filterlista)

# Max pooling

filterlistb = [5,5,5,5,5,5,5]
filterlist.append(filterlistb)
b = calculatefilter(a/2.0, filterlistb)

# Max pooling, Gate here

filterlistc = [7,7,7,7,7,7,7]
filterlist.append(filterlistc)
c = calculatefilter(b/2., filterlistc)

# Max pooling 

filterlistd = [5,5,5,5]
filterlist.append(filterlistd)
d = calculatefilter(c/2.0, filterlistd)


# Max pooling Gate Here

filterliste = [5,4,7]
filterlist.append(filterliste)
e = calculatefilter(d/2.0, filterliste)

print('Third')
print(e)

filterlist = []

filterlista = [3,3,3,3]
filterlist.append(filterlista)
a = calculatefilter(600, filterlista)

# Max pooling

filterlistb = [5,5,5,5,5,5,5]
filterlist.append(filterlistb)
b = calculatefilter(a/2.0, filterlistb)

# Max pooling

filterlistc = [7,7,7,7,7,7,7]
filterlist.append(filterlistc)
c = calculatefilter(b/2., filterlistc)

# Max pooling 

filterlistd = [5,5,5,5]
filterlist.append(filterlistd)
d = calculatefilter(c/2.0, filterlistd)


# Max pooling

filterliste = [5,4,7]
filterlist.append(filterliste)
e = calculatefilter(d/2.0, filterliste)

print('Fourth')
print(e)
'''
filterlist = []
# Gated
filterlista = [3,3,3,3]
filterlist.append(filterlista)
a = calculatefilter(600, filterlista)

# Max pooling

filterlistb = [5,5,5,5,5,5,5]
filterlist.append(filterlistb)
b = calculatefilter(a/2.0, filterlistb)

# Max pooling
# Gated
filterlistc = [7,7,7,7,7,7,7]
filterlist.append(filterlistc)
c = calculatefilter(b/2., filterlistc)

# Max pooling 

filterlistd = [5,5,5,5]
filterlist.append(filterlistd)
d = calculatefilter(c/2.0, filterlistd)


# Max pooling
# Gated


filterliste = [5,4,7]
filterlist.append(filterliste)
e = calculatefilter(d/2.0, filterliste)

print('Fifth')
print(e)

Fifth
2


In [ ]:
#Cascading deepness networks mem size
    
def channeling_rule(channelin, filterlist):        
  channellist = [channelin]
  for filter in filterlist:
    channellist.append(channellist[-1] + filter + 3)  

  return channellist 

In [10]:
#build graph
# gcnn2d,softmax,losscrossentropy
#config1 = [ {"0_frames":4,"0_0":(10,30,40,2)}, {"1_1":(10,2,2,8), "1_2":(10,2,1,16), "1_3":(10,4,1,32), "1_4":(10,8,1,64), "1_5":(10,8,1,64)},{"1_labels":2},{"2_null":a},{"3_labels":2,"3_learningrate":0.005}]
buildgraph = Builder(dtype=tf.float32,batch=48,datasize=(600,8), num_input = 10, channels=1)

buildgraph.get_directives('gcnn2d')
buildgraph.get_directives('softmax')
#buildgraph.set_archname('frame')
buildgraph.get_directives('reducemean')
buildgraph.get_directives('losscrossentropy')
pooling = [2,2,2,2]
for i in range(10):
    
    
  buildgraph.config_block_name(deepness=len(filterlist[0]),numblock=i)    
  for j in np.arange(len(filterlist)):    

    cout = 64 + j*20    
    if j != 0:
      buildgraph.config_block_name(deepness=len(filterlist[j]),numblock=i)  
    
    for z in np.arange(len(filterlist[j])):
        
      if z == 0 and j == 0: 
        buildgraph.build_graph_module('cnn2d', channels_out = 64,filter_size=(filterlist[j][0],8), isinput=True, lastnamescope=True, show_cgraph=False, verbose=False)  
      
      # z:= layer onde esta o gate dentro de um block de cnn, entre maxpoolings.
      # j:= número do block de cnn, onde esta o layer z de gate
      if z ==  (len(filterlist[j])-1) and (j == 4):
        buildgraph.build_graph_module('gcnn2d', channels_out = cout,filter_size=(filterlist[j][z],1), isinput=False, lastnamescope=True, show_cgraph=False, verbose=False)
      else:
        buildgraph.build_graph_module('cnn2d', channels_out = cout,filter_size=(filterlist[j][z],1), isinput=False, lastnamescope=True, show_cgraph=False, verbose=False)
        
    if j < len(filterlist) - 1:
      buildgraph.config_block_name(deepness=-1,numblock=i,pool=j)
      buildgraph.build_graph_module('maxpooling2d', poolsize = (pooling[j],1), isinput=False, lastnamescope=True, show_cgraph=False, verbose=False)             
    
  
  buildgraph.config_block_name(deepness=0,numblock=i)
  buildgraph.build_graph_module('softmax', num_labels=2, isinput=False, lastnamescope=True, show_cgraph=False)

'''
Avoiding the block reuse for awhile since tensorflow made it hard to keep
buildgraph.define_block('blocktest')
for i in np.arange(1,3):
  buildgraph.from_block('blocktest',tag='_'+str(i))
'''
buildgraph.config_block_name(deepness=0,numblock=0)
buildgraph.build_graph_module('reducemean', isinput=False, lastnamescope=True, show_cgraph=False)
buildgraph.config_block_name(deepness=0,numblock=0)
buildgraph.build_graph_module('losscrossentropy', num_labels = 2, batch=100, isinput=False, lastnamescope=True, show_cgraph=False)

W0826 16:14:40.801703 139844138194304 deprecation.py:323] From /home/penalvad/miniconda3/envs/scicomptf1_37/lib/python2.7/site-packages/tensorflow/python/ops/math_grad.py:1250: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [6]:
# Label Vector Feed
# dtrain[0][0][1]
ltrain = []
ltest = []

for i in range(14):    
  ltrain.append([0,1])   
  ltrain.append([1,0])
    
#for i in range(4):
  #ltest.append([0,1])    
  #ltest.append([1,0])

In [21]:
# Run Computation Graph

feed_dict = {}
for i in range(10):  
  feed_dict[buildgraph.signal_in[i]] = data_train[:6,:,8*i:8*(i+1),:]

feed_dict[buildgraph.l_vec] = ltrain[:6]
feed_dict[buildgraph.learning_rate] = 0.00001 

output = buildgraph.run_cgraph(feed_dict, op_to_run = buildgraph.namescopes['reducemean'][-1].name, number_of_runs = 3500, mode = 'minimize', new_session=True, output_log = True, auto_lr = False, verbose=True)

#for i in range(10):  
  #feed_dict[buildgraph.signal_in[i]] = data_train[:,:,5*i:5*(i+1),:]

#feed_dict[buildgraph.l_vec] = ltrain[:14]
#feed_dict[buildgraph.learning_rate] = 0.00001 

#acc = buildgraph.run_cgraph(feed_dict, op_to_run = buildgraph.namescopes['reducemean'][-1].name, number_of_runs = 1, mode = 'x', new_session=True, output_log = True, auto_lr = False, verbose=True)

#print('Tensor {}'.format(buildgraph.namescopes.gcnn2d_1[1].name))


Global Variables Initialized
('Step:', 0, ' Loss:', 0.6931620836257935)
('learning rate ', 9.999999747378752e-06)




[array([[0.25000054, 0.25001073],
       [0.25000024, 0.24999368],
       [0.24999306, 0.2500061 ],
       [0.25000623, 0.24998951]], dtype=float32)]
('Step:', 1, ' Loss:', 0.6931239366531372)
('learning rate ', 1.1853057047043611e-05)




[array([[0.24998942, 0.25001574],
       [0.25001097, 0.24998808],
       [0.24998192, 0.25001225],
       [0.2500177 , 0.24998394]], dtype=float32)]
('Step:', 2, ' Loss:', 0.693090558052063)
('learning rate ', 1.396034932737868e-05)




[array([[0.24998021, 0.25002193],
       [0.2500226 , 0.24998195],
       [0.24997127, 0.25001898],
       [0.25002593, 0.24997708]], dtype=float32)]
('Step:', 3, ' Loss:', 0.693057656288147)
3.5762786865234375e-05
('loss drop ', 0.00035762786865234375)
penalva
('learning rate ', 0.00011311367050139644)




[array([[0.24997039, 0.25002733],
       [0.25003415, 0.24997616],
       [0.24995911, 0.250024

[array([[0.17551482, 0.37700143],
       [0.29499248, 0.19322631],
       [0.18969211, 0.2332302 ],
       [0.3398006 , 0.19654208]], dtype=float32)]
('Step:', 28, ' Loss:', 0.48440611362457275)
0.049267321825027466
('loss drop ', 0.49267321825027466)
penalva
('learning rate ', 6.584787415394192e-07)




[array([[0.17539631, 0.3772743 ],
       [0.29513788, 0.19308528],
       [0.18953069, 0.23322484],
       [0.33993512, 0.19641557]], dtype=float32)]
('Step:', 29, ' Loss:', 0.4839860796928406)
0.007048711180686951
('loss drop ', 0.0704871118068695)
penalva
('learning rate ', 0.0002016305598161624)




[array([[0.17526697, 0.3775669 ],
       [0.2953091 , 0.19293277],
       [0.18935685, 0.23322237],
       [0.3400671 , 0.19627795]], dtype=float32)]
('Step:', 30, ' Loss:', 0.48352932929992676)
0.0039300620555877686
('loss drop ', 0.039300620555877686)
penalva
('learning rate ', 0.00010261122412476296)




[array([[0.14270745, 0.4723856 ],
       [0.3584478 , 0.14702779],
       [0.1271

('Step:', 54, ' Loss:', 1.1920928955078125e-07)
0.0012389002363306645
('loss drop ', 0.012389002363306645)
penalva
('learning rate ', 8.172507070104022e-06)
salvando modelo...




[array([[2.6354936e-08, 4.5887309e-01],
       [5.4156560e-01, 2.3026333e-08],
       [2.9240561e-11, 5.4112685e-01],
       [4.5843434e-01, 5.9425291e-09]], dtype=float32)]
('Step:', 55, ' Loss:', 1.1920928955078125e-07)
9.50707726588007e-06
('loss drop ', 9.50707726588007e-05)
penalva
('learning rate ', 0.00014334566213437615)




[array([[2.0938177e-08, 4.5945096e-01],
       [4.6303016e-01, 1.7871287e-08],
       [2.0852559e-11, 5.4054904e-01],
       [5.3696978e-01, 4.5097868e-09]], dtype=float32)]
('Step:', 56, ' Loss:', 1.1920928955078125e-07)
4.768376129504759e-07
('loss drop ', 4.768376129504759e-06)
penalva
('learning rate ', 9.225109946768127e-05)




[array([[3.6850825e-10, 4.6187449e-01],
       [5.0167519e-01, 1.9997692e-10],
       [5.3244020e-14, 5.3812551e-01],
       [4.9832481e-01, 3.358967

tfhelper.py:11: RuntimeWarning: invalid value encountered in double_scalars
  _ff = lambda ord,loss : 10.0*ord if loss//ord < 1.0/ord else ord
tfhelper.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  _ff = lambda ord,loss : 10.0*ord if loss//ord < 1.0/ord else ord
tfhelper.py:12: RuntimeWarning: invalid value encountered in double_scalars
  _fff = lambda ord,loss : ord/10.0 if loss//ord > 1.0/(ord/10.0) else ord
tfhelper.py:12: RuntimeWarning: divide by zero encountered in double_scalars
  _fff = lambda ord,loss : ord/10.0 if loss//ord > 1.0/(ord/10.0) else ord


[array([[1.7428062e-11, 4.4267362e-01],
       [5.0964737e-01, 7.3339121e-12],
       [6.8801739e-16, 5.5732644e-01],
       [4.9035263e-01, 9.1792914e-13]], dtype=float32)]
('Step:', 58, ' Loss:', 1.1920928955078125e-07)
0.0
('loss drop ', 0.0)
penalva
('learning rate ', 0.00028894166356260566)




[array([[5.7097737e-15, 4.8212987e-01],
       [4.1812021e-01, 1.0276861e-15],
       [9.3628695e-21, 5.1787007e-01],
       [5.8187979e-01, 6.7233697e-17]], dtype=float32)]
('Step:', 59, ' Loss:', 1.1920928955078125e-07)
0.0
('loss drop ', 0.0)
penalva
('learning rate ', 3.6730467866139164e-05)




[array([[1.4494734e-19, 4.9064049e-01],
       [4.0298072e-01, 4.7473745e-21],
       [3.3545929e-27, 5.0935948e-01],
       [5.9701931e-01, 1.2091289e-22]], dtype=float32)]
('Step:', 60, ' Loss:', 1.1920928955078125e-07)
0.0
('loss drop ', 0.0)
penalva
('learning rate ', 0.0002841511378181378)




[array([[4.0498068e-20, 4.9079061e-01],
       [4.0292636e-01, 9.2339751e-22],
       [5.3474263e-

[array([[0.        , 0.4999787 ],
       [0.41549483, 0.        ],
       [0.        , 0.50002134],
       [0.58450514, 0.        ]], dtype=float32)]
('Step:', 87, ' Loss:', 1.1920928955078125e-07)
0.0
('loss drop ', 0.0)
penalva
('learning rate ', 0.00023644265984412857)




[array([[0.        , 0.49998188],
       [0.41873503, 0.        ],
       [0.        , 0.5000181 ],
       [0.581265  , 0.        ]], dtype=float32)]
('Step:', 88, ' Loss:', 1.1920928955078125e-07)
0.0
('loss drop ', 0.0)
penalva
('learning rate ', 0.00017482998603620363)




[array([[0.        , 0.4999844 ],
       [0.4236477 , 0.        ],
       [0.        , 0.5000156 ],
       [0.57635236, 0.        ]], dtype=float32)]
('Step:', 89, ' Loss:', 1.1920928955078125e-07)
0.0
('loss drop ', 0.0)
penalva
('learning rate ', 1.2943269392974077e-05)




[array([[0.        , 0.49998608],
       [0.42994967, 0.        ],
       [0.        , 0.50001395],
       [0.57005036, 0.        ]], dtype=float32)]
('Step:', 90, ' Los

[array([[0.        , 0.49999627],
       [0.5188626 , 0.        ],
       [0.        , 0.50000376],
       [0.48113737, 0.        ]], dtype=float32)]
('Step:', 117, ' Loss:', 1.1920928955078125e-07)
0.0
('loss drop ', 0.0)
penalva
('learning rate ', 0.0001807891128754613)




[array([[0.        , 0.49999627],
       [0.5188924 , 0.        ],
       [0.        , 0.5000037 ],
       [0.48110762, 0.        ]], dtype=float32)]
('Step:', 118, ' Loss:', 1.1920928955078125e-07)
0.0
('loss drop ', 0.0)
penalva
('learning rate ', 0.00021219318927398115)




[array([[0.        , 0.49999633],
       [0.5189244 , 0.        ],
       [0.        , 0.5000037 ],
       [0.48107558, 0.        ]], dtype=float32)]
('Step:', 119, ' Loss:', 1.1920928955078125e-07)
0.0
('loss drop ', 0.0)
penalva
('learning rate ', 0.00014340306475985499)




[array([[0.        , 0.49999633],
       [0.51896536, 0.        ],
       [0.        , 0.5000037 ],
       [0.48103467, 0.        ]], dtype=float32)]
('Step:', 120, ' 

('Step:', 146, ' Loss:', 1.1920928955078125e-07)
0.0
('loss drop ', 0.0)
penalva
('learning rate ', 0.0002197624658547848)




[array([[0.        , 0.49999648],
       [0.51915526, 0.        ],
       [0.        , 0.5000035 ],
       [0.48084468, 0.        ]], dtype=float32)]
('Step:', 147, ' Loss:', 1.1920928955078125e-07)
0.0
('loss drop ', 0.0)
penalva
('learning rate ', 0.00014629704495161338)




[array([[0.        , 0.49999648],
       [0.5191543 , 0.        ],
       [0.        , 0.5000035 ],
       [0.4808457 , 0.        ]], dtype=float32)]
('Step:', 148, ' Loss:', 1.1920928955078125e-07)
0.0
('loss drop ', 0.0)
penalva
('learning rate ', 7.268824880254404e-05)




[array([[0.        , 0.49999648],
       [0.51915175, 0.        ],
       [0.        , 0.5000035 ],
       [0.48084822, 0.        ]], dtype=float32)]
('Step:', 149, ' Loss:', 1.1920928955078125e-07)
0.0
('loss drop ', 0.0)
penalva
('learning rate ', 0.00019893748462316652)




[array([[0.        , 0.49999648],
      

KeyboardInterrupt: 

In [14]:
# Recovering Model from .meta and .data. It uses .index and checkpoint to map variables and last checkpoint to the graph 
graph = tf.Graph()

with graph.as_default():
    
  saver = tf.train.import_meta_graph("./ckpt/model.meta", import_scope='')  
  feed_dict = {}  
  with tf.Session(graph=graph) as sess:
  # Restore variables from disk.
    
    saver.restore(sess, tf.train.latest_checkpoint('./ckpt/') )    
    feed_dict[sess.graph.get_tensor_by_name('signal_in:0')] = data_train[:4,:,:8,:]
    for i in np.arange(1,10):  
      feed_dict[sess.graph.get_tensor_by_name('signal_in_'+str(i)+':0')] = data_train[:4,:,8*i:8*(i+1),:]
    feed_dict[sess.graph.get_tensor_by_name('losscrossentropy/labels:0')] = ltrain[:4]
    feed_dict[sess.graph.get_tensor_by_name('learning_rate:0')] = 0.0001      
  #for op in graph.get_operations():
    #print(2*'\n')
    #print(op.name)  
    minimize_op = sess.graph.get_operation_by_name('losscrossentropy/Adam')
    loss = sess.graph.get_tensor_by_name('losscrossentropy/categorical_crossentropy/weighted_loss/value:0')
    acc = sess.graph.get_tensor_by_name('reducemean/Mean:0')
    for i in range(400):
      lossval,_ = sess.run([loss,minimize_op], feed_dict=feed_dict)
      accval = sess.run([acc], feed_dict=feed_dict)
      print('loss ',lossval)
      print('acc ', accval)
      print(2*'\n')

('loss ', 1.1920929e-07)
('acc ', [array([[[1.4177080e-09, 4.6343488e-01]],

       [[5.0779384e-01, 8.5192314e-10]],

       [[3.6752139e-13, 5.3656512e-01]],

       [[4.9220616e-01, 1.6435811e-10]]], dtype=float32)])



('loss ', 1.1920929e-07)
('acc ', [array([[[4.7876141e-11, 4.4873556e-01]],

       [[5.1053792e-01, 2.1010740e-11]],

       [[2.7575191e-15, 5.5126441e-01]],

       [[4.8946205e-01, 2.9173959e-12]]], dtype=float32)])



('loss ', 1.1920929e-07)
('acc ', [array([[[1.4066715e-12, 4.6015191e-01]],

       [[5.0720638e-01, 4.8993161e-13]],

       [[2.1505860e-17, 5.3984803e-01]],

       [[4.9279362e-01, 5.0224480e-14]]], dtype=float32)])



('loss ', 1.1920929e-07)
('acc ', [array([[[5.2144324e-14, 4.7868791e-01]],

       [[4.4149208e-01, 1.2445652e-14]],

       [[2.1133800e-19, 5.2131212e-01]],

       [[5.5850798e-01, 9.9004906e-16]]], dtype=float32)])



('loss ', 1.1920929e-07)
('acc ', [array([[[1.9425400e-15, 4.8334616e-01]],

       [[4.1406855e-01, 2.98658

('loss ', 1.1920929e-07)
('acc ', [array([[[1.6213949e-34, 4.9907365e-01]],

       [[4.2065090e-01, 0.0000000e+00]],

       [[0.0000000e+00, 5.0092638e-01]],

       [[5.7934910e-01, 6.3067164e-38]]], dtype=float32)])



('loss ', 1.1920929e-07)
('acc ', [array([[[1.2563988e-34, 4.9908608e-01]],

       [[4.2288294e-01, 0.0000000e+00]],

       [[0.0000000e+00, 5.0091392e-01]],

       [[5.7711709e-01, 5.7584970e-38]]], dtype=float32)])



('loss ', 1.1920929e-07)
('acc ', [array([[[9.9716279e-35, 4.9909773e-01]],

       [[4.2512283e-01, 0.0000000e+00]],

       [[0.0000000e+00, 5.0090224e-01]],

       [[5.7487714e-01, 5.3004204e-38]]], dtype=float32)])



('loss ', 1.1920929e-07)
('acc ', [array([[[8.0810019e-35, 4.9910945e-01]],

       [[4.2735377e-01, 0.0000000e+00]],

       [[0.0000000e+00, 5.0089055e-01]],

       [[5.7264626e-01, 4.9182516e-38]]], dtype=float32)])



('loss ', 1.1920929e-07)
('acc ', [array([[[6.6657251e-35, 4.9911737e-01]],

       [[4.2962274e-01, 0.00000

('loss ', 1.1920929e-07)
('acc ', [array([[[1.1024935e-35, 4.9919924e-01]],

       [[4.5775977e-01, 0.0000000e+00]],

       [[0.0000000e+00, 5.0080073e-01]],

       [[5.4224026e-01, 2.4789211e-38]]], dtype=float32)])



('loss ', 1.1920929e-07)
('acc ', [array([[[1.09508075e-35, 4.99199629e-01]],

       [[4.57908332e-01, 0.00000000e+00]],

       [[0.00000000e+00, 5.00800371e-01]],

       [[5.42091668e-01, 2.47047266e-38]]], dtype=float32)])



('loss ', 1.1920929e-07)
('acc ', [array([[[1.08880645e-35, 4.99199629e-01]],

       [[4.57905948e-01, 0.00000000e+00]],

       [[0.00000000e+00, 5.00800371e-01]],

       [[5.42094052e-01, 2.46685647e-38]]], dtype=float32)])



('loss ', 1.1920929e-07)
('acc ', [array([[[1.0826554e-35, 4.9920025e-01]],

       [[4.5811337e-01, 0.0000000e+00]],

       [[0.0000000e+00, 5.0079978e-01]],

       [[5.4188663e-01, 2.4614564e-38]]], dtype=float32)])



('loss ', 1.1920929e-07)
('acc ', [array([[[1.07804864e-35, 4.99200255e-01]],

       [[4.58

('loss ', 1.1920929e-07)
('acc ', [array([[[1.0327783e-35, 4.9920216e-01]],

       [[4.5882207e-01, 0.0000000e+00]],

       [[0.0000000e+00, 5.0079787e-01]],

       [[5.4117793e-01, 2.4286728e-38]]], dtype=float32)])



('loss ', 1.1920929e-07)
('acc ', [array([[[1.03252406e-35, 4.99201775e-01]],

       [[4.58778232e-01, 0.00000000e+00]],

       [[0.00000000e+00, 5.00798225e-01]],

       [[5.41221738e-01, 2.43044317e-38]]], dtype=float32)])



('loss ', 1.1920929e-07)
('acc ', [array([[[1.0325497e-35, 4.9920234e-01]],

       [[4.5884567e-01, 0.0000000e+00]],

       [[0.0000000e+00, 5.0079763e-01]],

       [[5.4115433e-01, 2.4286775e-38]]], dtype=float32)])



('loss ', 1.1920929e-07)
('acc ', [array([[[1.03235345e-35, 4.99201775e-01]],

       [[4.58797693e-01, 0.00000000e+00]],

       [[0.00000000e+00, 5.00798225e-01]],

       [[5.41202307e-01, 2.42866380e-38]]], dtype=float32)])



('loss ', 1.1920929e-07)
('acc ', [array([[[1.0324868e-35, 4.9920234e-01]],

       [[4.5882

('loss ', 1.1920929e-07)
('acc ', [array([[[1.03214474e-35, 4.99202907e-01]],

       [[4.58810896e-01, 0.00000000e+00]],

       [[0.00000000e+00, 5.00797093e-01]],

       [[5.41189075e-01, 2.42691386e-38]]], dtype=float32)])



('loss ', 1.1920929e-07)
('acc ', [array([[[1.0322300e-35, 4.9920195e-01]],

       [[4.5882854e-01, 0.0000000e+00]],

       [[0.0000000e+00, 5.0079805e-01]],

       [[5.4117143e-01, 2.4274828e-38]]], dtype=float32)])



('loss ', 1.1920929e-07)
('acc ', [array([[[1.0322553e-35, 4.9920177e-01]],

       [[4.5884126e-01, 0.0000000e+00]],

       [[0.0000000e+00, 5.0079823e-01]],

       [[5.4115874e-01, 2.4268858e-38]]], dtype=float32)])



('loss ', 1.1920929e-07)
('acc ', [array([[[1.0320925e-35, 4.9920234e-01]],

       [[4.5881671e-01, 0.0000000e+00]],

       [[0.0000000e+00, 5.0079763e-01]],

       [[5.4118329e-01, 2.4274923e-38]]], dtype=float32)])



('loss ', 1.1920929e-07)
('acc ', [array([[[1.03214474e-35, 4.99201596e-01]],

       [[4.58883107e-

...

('loss ', 1.1920929e-07)
('acc ', [array([[[1.0322153e-35, 4.9920273e-01]],

       [[4.5887175e-01, 0.0000000e+00]],

       [[0.0000000e+00, 5.0079727e-01]],

       [[5.4112828e-01, 2.4269091e-38]]], dtype=float32)])



('loss ', 1.1920929e-07)
('acc ', [array([[[1.0322153e-35, 4.9920273e-01]],

       [[4.5887175e-01, 0.0000000e+00]],

       [[0.0000000e+00, 5.0079727e-01]],

       [[5.4112828e-01, 2.4269091e-38]]], dtype=float32)])



('loss ', 1.1920929e-07)
('acc ', [array([[[1.0322153e-35, 4.9920273e-01]],

       [[4.5887175e-01, 0.0000000e+00]],

       [[0.0000000e+00, 5.0079727e-01]],

       [[5.4112828e-01, 2.4269091e-38]]], dtype=float32)])



('loss ', 1.1920929e-07)
('acc ', [array([[[1.0322153e-35, 4.9920273e-01]],

       [[4.5887175e-01, 0.0000000e+00]],

       [[0.0000000e+00, 5.0079727e-01]],

       [[5.4112828e-01, 2.4269091e-38]]], dtype=float32)])



('loss ', 1.1920929e-07)
('acc ', [array([[[1.0322153e-35, 4.9920273e-01]],

       [[4.5887175e-01, 0.00000

('loss ', 1.1920929e-07)
('acc ', [array([[[1.0322153e-35, 4.9920273e-01]],

       [[4.5887175e-01, 0.0000000e+00]],

       [[0.0000000e+00, 5.0079727e-01]],

       [[5.4112828e-01, 2.4269091e-38]]], dtype=float32)])



('loss ', 1.1920929e-07)
('acc ', [array([[[1.0322153e-35, 4.9920273e-01]],

       [[4.5887175e-01, 0.0000000e+00]],

       [[0.0000000e+00, 5.0079727e-01]],

       [[5.4112828e-01, 2.4269091e-38]]], dtype=float32)])



('loss ', 1.1920929e-07)
('acc ', [array([[[1.0322153e-35, 4.9920273e-01]],

       [[4.5887175e-01, 0.0000000e+00]],

       [[0.0000000e+00, 5.0079727e-01]],

       [[5.4112828e-01, 2.4269091e-38]]], dtype=float32)])



('loss ', 1.1920929e-07)
('acc ', [array([[[1.0322153e-35, 4.9920273e-01]],

       [[4.5887175e-01, 0.0000000e+00]],

       [[0.0000000e+00, 5.0079727e-01]],

       [[5.4112828e-01, 2.4269091e-38]]], dtype=float32)])



('loss ', 1.1920929e-07)
('acc ', [array([[[1.0322153e-35, 4.9920273e-01]],

       [[4.5887175e-01, 0.00000

('loss ', 1.1920929e-07)
('acc ', [array([[[1.0322153e-35, 4.9920273e-01]],

       [[4.5887175e-01, 0.0000000e+00]],

       [[0.0000000e+00, 5.0079727e-01]],

       [[5.4112828e-01, 2.4269091e-38]]], dtype=float32)])



('loss ', 1.1920929e-07)
('acc ', [array([[[1.0322153e-35, 4.9920273e-01]],

       [[4.5887175e-01, 0.0000000e+00]],

       [[0.0000000e+00, 5.0079727e-01]],

       [[5.4112828e-01, 2.4269091e-38]]], dtype=float32)])



('loss ', 1.1920929e-07)
('acc ', [array([[[1.0322153e-35, 4.9920273e-01]],

       [[4.5887175e-01, 0.0000000e+00]],

       [[0.0000000e+00, 5.0079727e-01]],

       [[5.4112828e-01, 2.4269091e-38]]], dtype=float32)])



('loss ', 1.1920929e-07)
('acc ', [array([[[1.0322153e-35, 4.9920273e-01]],

       [[4.5887175e-01, 0.0000000e+00]],

       [[0.0000000e+00, 5.0079727e-01]],

       [[5.4112828e-01, 2.4269091e-38]]], dtype=float32)])



('loss ', 1.1920929e-07)
('acc ', [array([[[1.0322153e-35, 4.9920273e-01]],

       [[4.5887175e-01, 0.00000

('loss ', 1.1920929e-07)
('acc ', [array([[[1.0322153e-35, 4.9920273e-01]],

       [[4.5887175e-01, 0.0000000e+00]],

       [[0.0000000e+00, 5.0079727e-01]],

       [[5.4112828e-01, 2.4269091e-38]]], dtype=float32)])



('loss ', 1.1920929e-07)
('acc ', [array([[[1.0322153e-35, 4.9920273e-01]],

       [[4.5887175e-01, 0.0000000e+00]],

       [[0.0000000e+00, 5.0079727e-01]],

       [[5.4112828e-01, 2.4269091e-38]]], dtype=float32)])



('loss ', 1.1920929e-07)
('acc ', [array([[[1.0322153e-35, 4.9920273e-01]],

       [[4.5887175e-01, 0.0000000e+00]],

       [[0.0000000e+00, 5.0079727e-01]],

       [[5.4112828e-01, 2.4269091e-38]]], dtype=float32)])



('loss ', 1.1920929e-07)
('acc ', [array([[[1.0322153e-35, 4.9920273e-01]],

       [[4.5887175e-01, 0.0000000e+00]],

       [[0.0000000e+00, 5.0079727e-01]],

       [[5.4112828e-01, 2.4269091e-38]]], dtype=float32)])



('loss ', 1.1920929e-07)
('acc ', [array([[[1.0322153e-35, 4.9920273e-01]],

       [[4.5887175e-01, 0.00000

('loss ', 1.1920929e-07)
('acc ', [array([[[1.0322153e-35, 4.9920273e-01]],

       [[4.5887175e-01, 0.0000000e+00]],

       [[0.0000000e+00, 5.0079727e-01]],

       [[5.4112828e-01, 2.4269091e-38]]], dtype=float32)])



('loss ', 1.1920929e-07)
('acc ', [array([[[1.0322153e-35, 4.9920273e-01]],

       [[4.5887175e-01, 0.0000000e+00]],

       [[0.0000000e+00, 5.0079727e-01]],

       [[5.4112828e-01, 2.4269091e-38]]], dtype=float32)])



('loss ', 1.1920929e-07)
('acc ', [array([[[1.0322153e-35, 4.9920273e-01]],

       [[4.5887175e-01, 0.0000000e+00]],

       [[0.0000000e+00, 5.0079727e-01]],

       [[5.4112828e-01, 2.4269091e-38]]], dtype=float32)])



('loss ', 1.1920929e-07)
('acc ', [array([[[1.0322153e-35, 4.9920273e-01]],

       [[4.5887175e-01, 0.0000000e+00]],

       [[0.0000000e+00, 5.0079727e-01]],

       [[5.4112828e-01, 2.4269091e-38]]], dtype=float32)])



('loss ', 1.1920929e-07)
('acc ', [array([[[1.0322153e-35, 4.9920273e-01]],

       [[4.5887175e-01, 0.00000